## 10 · Helper: Ray-Prepared DataLoaders  
Wrap the DataLoader with `prepare_data_loader`. Ray injects a `DistributedSampler`, pins the loader to the correct GPU, and handles worker-rank bookkeeping. Thanks to this helper, your training loop can stay blissfully unaware of distributed details.

In [ ]:
# 10. Define helper to create prepared DataLoader
from torch.utils.data.distributed import DistributedSampler

def build_dataloader(parquet_path: str, batch_size: int, shuffle=True):
    dataset = Food101Dataset(parquet_path, transform=transform)

    # Add a DistributedSampler to shard data across workers
    sampler = DistributedSampler(
        dataset,
        num_replicas=train.get_context().get_world_size(),
        rank=train.get_context().get_world_rank(),
        shuffle=shuffle,
        drop_last=shuffle,
    )

    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        sampler=sampler,
        num_workers=2,
    )
    return prepare_data_loader(loader)